In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import rasterio
from rasterio.plot import show
import rioxarray as rxr
import xarray as xr
from shapely.geometry import mapping
import numpy as np
import matplotlib.colors
from matplotlib import cm

In [ ]:
filepath = ' '  # edit this!!
filename = ' '  # add file name!

In [ ]:
os.chdir(filepath)
data = rasterio.open(filename)

In [ ]:
blues = cm.get_cmap('Blues', 256)
newcolors = blues(np.linspace(0, 1.5, 256))
pink = np.array([248/256, 24/256, 148/256, 1])
white = np.array([1,1,1,0])
newcolors[:10, :] = white
newcmp = matplotlib.colors.ListedColormap(newcolors)

kwargs = {"cmap": newcmp, "vmax": 1}

fig, ax = plt.subplots(figsize=(15, 10))
swe_plot = show((data,1),ax=ax,vmin=0.1,**kwargs)
colorbar = fig.colorbar(swe_plot,ax=ax)
ax.set_xlabel('easting [m]')
ax.set_ylabel('northing [m]')
fig.colorbar(cm.ScalarMappable(norm=None, cmap=newcmp), ax=ax)
plt.show()